# 앙상블 학습
## 앙상블 학습의 개요
앙상블(Ensemble)을 통한 분류는 여러 개의 분류기(Classifier)를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법  
앙상블의 유형:
- 보팅(Voting)
- 배깅(Bagging)
- 부스팅(Boosting)
- 스태킹(Stacking)  

보팅과 배깅은 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식  
- 차이점:
    - 보팅: 서로 다른 알고리즘을 결합함
    - 배깅: 각각의 분류기가 모두 같은 알고리즘이지만, 데이터 샘플링을 다르게 가져가면서 보팅을 수행  
    
배깅 학습 시 개별 분류기에 할당된 학습 데이터는 원본 학습 데이터를 샘플링하는데 개별 Classifier에게 데이터를 샘플링해서 추출하는 방식이 부트스트래핑(Bootstrapping)  
개별 분류기가 부트스트래핑 방식으로 샘플링된 데이터 세트에 대해서 학습을 통해 개별적인 예측을 수행한 결과를 보팅을 통해서 최종 예측 결과를 선정하는 방식이 배깅 앙상블 방식  
배깅 방식은 중첩을 허용

![image](https://user-images.githubusercontent.com/70187490/128851348-c69e5654-9f1e-43b3-b566-4fb66141164a.png)

부스팅:
- 여러 개의 분류기가 순차적으로 학습을 수행하되,
- 앞의 분류기가 예측이 틀린 데이터에 대해서는 다음 분류기에게 가중치를 부여함
- 이러한 학습과 예측이 진행됨
- 계속해서 분류기에게 가중치를 부스팅하면서 학습을 진행하기에 부스팅 방식이라 불림
- 그래디언트 부스트, XGBoost(eXtra Gradient Boost), LightGBM(Light Gradient Boost)  

스태킹: 여러 가지 다른 모델의 예측 결괏값을 다시 학습 데이터로 만들어서 다른 모델(메타 모델)로 재학습시켜 결과를 예측

## 보팅 유형 - 하드 보팅(Hard Voting)과 소프트 보팅(Soft Voting)

![image](https://user-images.githubusercontent.com/70187490/128854674-01529d75-01ec-43b5-a00a-47993518192f.png)

> 하드 보팅은 예측 한 결괏값들중 다수의 분류기가 결정한 예측값을 최종 보팅 결괏값으로 선정  


> 소프트 보팅은 분류기들의 레이블 값 결정 확률을 모두 더하고 이를 평균해서 이들 중 확률이 가장 높은 레이블 값을 최종 보팅 결괏값으로 선정  


일반적으로 소프트 보팅이 보팅 방법으로 적용됨

In [58]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [62]:
# 개별 모델은 로지스틱 회귀와 KNN임
lr_clf = LogisticRegression(max_iter=2887)
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier( estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    test_size=0.2, random_state=156)

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred=vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred=classifier.predict(X_test)
    class_name=classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

Voting 분류기 정확도: 0.9474
LogisticRegression 정확도: 0.9649
KNeighborsClassifier 정확도: 0.9386


- 보팅으로 여러 개의 기반 분류기를 결합한다고 해서 무조건 기반 분류기보다 예측 성능이 향상되는 것은아님
- 보팅과 스태킹 등은 서로 다른 알고리즘을 기반으로 하고 있지만,
- 배깅과 부스팅은 대부분 결정 트리 알고리즘을 기반으로 함
- 결정 트리 알고리즘은 과적합이 발생할 위험이 크지만 앙상블 학습은 결정 트리 알고리즘을 결합함으로써 그 단점을 극복함
- 결정 트리의 장점을 그대로 취하고 단점은 보완하면서 편향-분산 트레이드오프의 효과를 극대화